In [1]:
pip install awswrangler

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-surprise

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate
import boto3
from datetime import datetime, timedelta
import awswrangler as wr
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 500) 

In [4]:
df = wr.s3.read_parquet(f's3://test-uala-arg-datalake-aiml-recommendations/data/stage/all_data_v2/')
df.rename(columns={"nu_sin_categoria_aprob":"nu_compras_otras_categorias_aprob"}, inplace=True)

# Limpieza de datos

In [5]:
df[['cobros','cuotificaciones','promociones','transferencia_c2c','transferencia_cvu']] = df[['cobros','cuotificaciones','promociones','transferencia_c2c','transferencia_cvu']].fillna(0)

In [6]:
df = df.dropna()

In [7]:
df['dt'] = pd.to_datetime(df['dt'])

In [8]:
df = df.rename({'dt':'periodo'}, axis=1)

The variable user account age is created:

In [9]:
# Last timeframe available
max_periodo = df.periodo.max()
max_periodo = pd.to_datetime(max_periodo)

In [10]:
df.fecha_alta = pd.to_datetime(df.fecha_alta)
df.fecha_alta = np.where(df.fecha_alta > max_periodo , max_periodo, df.fecha_alta.dt.date)

In [11]:
df['antiguedad']= max_periodo - df.fecha_alta

In [12]:
df['antiguedad'] = df['antiguedad'].dt.days

# Transformacion de datos

In [13]:
	Cust_Id	Movie_Id	Rating	Date
0	1488844	1	3	2005-09-06
1	822109	1	5	2005-05-13

SyntaxError: invalid syntax (<ipython-input-13-2496a664973a>, line 1)

## Seleccion de columnas y filtro de fecha

In [16]:
variables=['account_id',
     'periodo',
 'nu_cashin_prestamos_qty',
 'nu_cashin_adquirencia_qty',
 'nu_investments_deposit_aprob',
 'nu_telerecargas_carga_aprob',
 'nu_compras_otras_categorias_aprob',
 'nu_compras_aprob',
 'nu_entretenimiento_aprob',
 'nu_servicios_débitos_automaticos_aprob',
 'nu_supermercados_alimentos_aprob']

In [17]:
df_=df[df['periodo'].isin(['2021-03','2021-02','2021-01'])].copy()

In [18]:
df_=df_.loc[:,variables]

In [19]:
df_.head()

,account_id,periodo,nu_cashin_prestamos_qty,nu_cashin_adquirencia_qty,nu_investments_deposit_aprob,nu_telerecargas_carga_aprob,nu_compras_otras_categorias_aprob,nu_compras_aprob,nu_entretenimiento_aprob,nu_servicios_débitos_automaticos_aprob,nu_supermercados_alimentos_aprob
0,8855f17f-5370-4d27-98cc-03a7f2c28dda,2021-03-01,0,0,0,0,0,0,0,0,0
1,e0c1285a-81ca-4b9e-a801-df9f50d7ff59,2021-03-01,0,0,0,0,0,0,0,0,0
2,77c157b8-a920-44fd-bdc0-b8232dc28475,2021-03-01,0,0,0,0,0,0,0,0,0
3,7ea81038-6419-4b39-9735-e70da1f13607,2021-03-01,0,0,0,0,0,0,0,0,0
4,feab2f3b-d5f4-48fc-b678-170bd9c1d839,2021-03-01,0,0,0,0,0,0,0,0,0


### Generamos un "rating" para los productos

Por account_id vamos a obtener una proporción de uso de cada producto sobre el total de transacciones realizadas con esos productos.

In [20]:
variables = ['nu_cashin_prestamos_qty',
 'nu_cashin_adquirencia_qty',
 'nu_investments_deposit_aprob',
 'nu_telerecargas_carga_aprob',
 'nu_compras_otras_categorias_aprob',
 'nu_compras_aprob',
 'nu_entretenimiento_aprob',
 'nu_servicios_débitos_automaticos_aprob',
 'nu_supermercados_alimentos_aprob']

In [22]:
tmp = (df_
 .groupby(['account_id'])
 .agg('sum')
 .reset_index()
)

tmp['total']= tmp[variables].sum(axis=1)
tmp['rt_cashin_prestamos_qty'] = tmp['nu_cashin_prestamos_qty']/tmp['total']
tmp['rt_cashin_adquirencia_qty'] = tmp['nu_cashin_adquirencia_qty']/tmp['total']
tmp['rt_investments_deposit_aprob'] = tmp['nu_investments_deposit_aprob']/tmp['total']
tmp['rt_telerecargas_carga_aprob'] = tmp['nu_compras_otras_categorias_aprob']/tmp['total']
tmp['rt_compras_aprob'] = tmp['nu_compras_aprob']/tmp['total']
tmp['rt_entretenimiento_aprob'] = tmp['nu_entretenimiento_aprob']/tmp['total']
tmp['rt_servicios_débitos_automaticos_aprob'] = tmp['nu_servicios_débitos_automaticos_aprob']/tmp['total']
tmp['rt_supermercados_alimentos_aprob'] = tmp['nu_supermercados_alimentos_aprob']/tmp['total']

tmp = tmp.fillna(0)
tmp.head(2)

,account_id,nu_cashin_prestamos_qty,nu_cashin_adquirencia_qty,nu_investments_deposit_aprob,nu_telerecargas_carga_aprob,nu_compras_otras_categorias_aprob,nu_compras_aprob,nu_entretenimiento_aprob,nu_servicios_débitos_automaticos_aprob,nu_supermercados_alimentos_aprob,total,rt_cashin_prestamos_qty,rt_cashin_adquirencia_qty,rt_investments_deposit_aprob,rt_telerecargas_carga_aprob,rt_compras_aprob,rt_entretenimiento_aprob,rt_servicios_débitos_automaticos_aprob,rt_supermercados_alimentos_aprob
0,000005b1-6bd9-473e-aecf-3c23739ef825,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,00000750-eb3a-4dc1-a91e-701ac44aeec1,0,0,11,0,16,0,0,0,0,27.0,0.0,0.0,0.407407,0.592593,0.0,0.0,0.0,0.0


In [23]:
tmp.describe()

,nu_cashin_prestamos_qty,nu_cashin_adquirencia_qty,nu_investments_deposit_aprob,nu_telerecargas_carga_aprob,nu_compras_otras_categorias_aprob,nu_compras_aprob,nu_entretenimiento_aprob,nu_servicios_débitos_automaticos_aprob,nu_supermercados_alimentos_aprob,total,rt_cashin_prestamos_qty,rt_cashin_adquirencia_qty,rt_investments_deposit_aprob,rt_telerecargas_carga_aprob,rt_compras_aprob,rt_entretenimiento_aprob,rt_servicios_débitos_automaticos_aprob,rt_supermercados_alimentos_aprob
count,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06,1.215340e+06
mean,2.767950e-03,2.411671e-03,4.247758e-01,4.922754e-01,2.943748e+00,1.631774e-01,2.102630e-01,4.409030e-01,1.970296e-01,4.877352e+00,8.064496e-05,9.654437e-05,1.637940e-02,1.210678e-01,8.955836e-03,1.644001e-02,2.960299e-02,8.409118e-03
std,1.560796e-01,1.320619e-01,6.402027e+00,6.697775e+00,2.486822e+01,1.554317e+00,3.573281e+00,2.566715e+00,1.701075e+00,3.261764e+01,4.078556e-03,5.667616e-03,7.023873e-02,2.684682e-01,6.144721e-02,9.674109e-02,1.201933e-01,5.782445e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,5.200000e+01,4.900000e+01,4.667000e+03,2.865000e+03,7.786000e+03,3.320000e+02,1.037000e+03,4.050000e+02,2.080000e+02,1.250400e+04,1.000000e+00,1.000000e+00,5.000000e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


Convertimos los ratios o proporciones de transacciones, en ratings o puntaje del 1 al 5 para cada producto y cliente:

In [24]:
tmp['rt_cashin_prestamos_qty'] = tmp['rt_cashin_prestamos_qty']*5
tmp['rt_cashin_adquirencia_qty'] = tmp['rt_cashin_adquirencia_qty']*5
tmp['rt_investments_deposit_aprob'] = tmp['rt_investments_deposit_aprob']*5
tmp['rt_telerecargas_carga_aprob'] = tmp['rt_telerecargas_carga_aprob']*5
tmp['rt_compras_aprob'] = tmp['rt_compras_aprob']*5
tmp['rt_entretenimiento_aprob'] = tmp['rt_entretenimiento_aprob']*5 
tmp['rt_servicios_débitos_automaticos_aprob'] = tmp['rt_servicios_débitos_automaticos_aprob']*5
tmp['rt_supermercados_alimentos_aprob'] = tmp['rt_supermercados_alimentos_aprob']*5

In [25]:
tmp.head(10)

,account_id,nu_cashin_prestamos_qty,nu_cashin_adquirencia_qty,nu_investments_deposit_aprob,nu_telerecargas_carga_aprob,nu_compras_otras_categorias_aprob,nu_compras_aprob,nu_entretenimiento_aprob,nu_servicios_débitos_automaticos_aprob,nu_supermercados_alimentos_aprob,total,rt_cashin_prestamos_qty,rt_cashin_adquirencia_qty,rt_investments_deposit_aprob,rt_telerecargas_carga_aprob,rt_compras_aprob,rt_entretenimiento_aprob,rt_servicios_débitos_automaticos_aprob,rt_supermercados_alimentos_aprob
0,000005b1-6bd9-473e-aecf-3c23739ef825,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,00000750-eb3a-4dc1-a91e-701ac44aeec1,0,0,11,0,16,0,0,0,0,27.0,0.0,0.0,2.037037,2.962963,0.0,0.0,0.000000,0.0
2,00002479-2ed2-4c6d-9cc1-2e827857bf3d,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,000043cd-83ff-4a51-9ca3-520f01bf73e5,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,00004a06-2d8a-4951-bab1-be61cd74aac9,0,0,0,5,6,0,0,1,0,12.0,0.0,0.0,0.000000,2.500000,0.0,0.0,0.416667,0.0
5,000067b3-bc3f-4f24-88e0-0b879dcb0340,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
6,0000680e-53f6-44d1-8c61-53ff6f0a5d13,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
7,0000ae3a-8099-4f4a-9b85-2da5a0bad8fe,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
8,0000ba0e-4f1e-45d8-bad6-7a694b8411fe,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
9,0000c1d9-b9da-4aeb-94a5-c538b2a6d05e,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0


### Hacemos pivot del df final

In [ ]:
data_pivot = pd.melt(df_, 
                     id_vars=['account_id'], 
                     var_name='producto', 
                     value_name='value')

In [ ]:
data_pivot=data_pivot[data_pivot.value > 0]

In [ ]:
data_pivot.head(20)

In [ ]:
data_pivot['value'] = data_pivot['value'].astype(float)

In [ ]:
data_pivot.dtypes

# Modelado

In [ ]:
reader = Reader()
data = Dataset.load_from_df(data_pivot, reader)
svd = SVD()
# Run 5-fold cross-validation and print results
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

## Recomendacion a un cliente de prueba

In [ ]:
data_pivot[(data_pivot['account_id'] == '000b3d7b-ca88-474c-90e6-a16a25a29bf7')]

In [ ]:
df_cliente = data_pivot[(data_pivot['account_id'] == '000b3d7b-ca88-474c-90e6-a16a25a29bf7')]
#df_cliente = df_cliente.set_index('Movie_Id')
#df_cliente = df_cliente.join(df_title)['Name']
#df_cliente.head(df_cliente.shape[0])

## Refit

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
prueba = df_cliente.copy()
prueba['Estimate_Score'] = prueba['producto'].apply(lambda x: svd.predict(1, x).est)
prueba = prueba.sort_values(by=['Estimate_Score'], ascending=False)

In [ ]:
prueba

In [ ]:
df_cliente